In [ ]:
from groq import Groq
import datetime
import json

client = Groq(api_key="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")

def extract_tasks_from_mom(mom_text, meeting_date=None):
    """
    Extract actionable tasks from meeting minutes using Groq's LLM API.
    Returns JSON array of tasks with title, description, deadline, tags, and priority.
    
    Args:
        mom_text (str): The meeting transcript text
        meeting_date (str, optional): The meeting date in YYYY-MM-DD format
    """
    # If no date provided, use today's date
    if not meeting_date:
        meeting_date = datetime.datetime.now().strftime('%Y-%m-%d')
    
    prompt = f"""
    Extract actionable tasks from the following meeting minutes recorded on {meeting_date}.
    
    For each task, provide:
    1. A clear, concise title that summarizes the action required
    2. A detailed description of what needs to be done
    3. The person/people responsible for the task (include actual names from the transcript)
    4. When the task should be completed (deadline/timeframe)
    5. Tags that include people (prefix with @), categories (Finance, IT, Marketing, etc.), and priority levels
    
    Output should be ONLY a valid JSON array with no additional text, comments, or explanations:
    [
      {{
        "title": "Task title",
        "description": "Detailed description of what needs to be done",
        "deadline": "YYYY-MM-DD",  # Use a clear date format or null if no deadline
        "tags": ["@PersonName", "Department", "Priority"],
        "priority": number  # 1 to 10, where 1 is highest priority and 10 is lowest
      }}
    ]
    
    Remember:
    - Do NOT make up or invent tasks that aren't clearly mentioned in the transcript
    - Do NOT include tasks that are already completed
    - Only extract clear actions that someone needs to take
    - Be specific with names and deadlines mentioned in the transcript
    - Format the JSON correctly with no trailing commas
    - Today's date is {meeting_date}, so interpret relative dates accordingly
    
    Meeting transcript:
    {mom_text}
    """
    
    # For non-streaming version (better for backend use)
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,  # Lower temperature for more predictable outputs
        max_completion_tokens=2048,
        top_p=1,
        stream=False,
        stop=None,
    )
    
    return completion.choices[0].message.content

APIConnectionError: Connection error.

In [ ]:
# Example usage
mom_text = """
Meeting: Project Status Update
Date: May 30, 2025

Discussion:
The marketing campaign is almost ready, but we're still waiting on budget approval. It needs to be signed off by next week to keep everything on track. The budget allocation for Q3 is in progress, and Rahul is working on closing that. On the IT side, the server migration is scheduled to wrap up by Friday, and the team is also gearing up to implement the new CRM system by the end of the month. HR is preparing onboarding materials for the new hires, and they're also finalizing the training schedule to ensure a smooth start. Meanwhile, there's progress on the communications front—Pooja is coordinating with the external agency to lock in the social media strategy for the upcoming campaign. Finance is currently focused on compiling the quarterly report, which will give us better insight into overall performance. Everything's moving, but some items are time-sensitive and need quick decisions to stay aligned with the plan.
"""

# Test non-streaming version with today's date
today = datetime.datetime.now().strftime('%Y-%m-%d')
print(f"Processing with date: {today}")

tasks_json = extract_tasks_from_mom(mom_text, today)
print(tasks_json)

# Parse and use the JSON
tasks = json.loads(tasks_json)
print("\nExtracted", len(tasks), "tasks.")

# Example of converting a task to a note
if tasks:
    example_note = {
        "title": tasks[0]["title"],
        "description": tasks[0]["description"],
        "tags": tasks[0]["tags"],
        "deadline": tasks[0]["deadline"],
        "assigned_to": next((tag for tag in tasks[0]["tags"] if tag.startswith('@')), None),
        "priority": tasks[0]["priority"] <= 3  # High priority if 3 or less
    }
    print("\nExample note object:", json.dumps(example_note, indent=2))

NameError: name 'tasks_json' is not defined

# Text-based Task Extraction Flow

This notebook demonstrates the simplified workflow for extracting tasks from meeting transcripts:

1. **Get Transcript** - Instead of processing audio, we directly work with text transcripts 
2. **Process with Groq LLM** - Send the transcript to Groq for task extraction
3. **Format Results** - Get structured task data with titles, descriptions, deadlines and assignments

This approach:
- Eliminates heavy audio processing dependencies
- Works with any source of transcription (frontend, third-party services, etc.)
- Focuses on the core NLP task of identifying actionable items
- Simplifies system architecture and deployment

In [ ]:
# Demonstrate the direct text processing workflow with a manual transcription

def process_manual_transcription():
    """Process a manually entered meeting transcription"""
    
    # In a real application, this would come from the frontend or a file
    transcript = """
    Meeting: Sprint Planning - June 15, 2023
    
    Project Manager: Let's outline our tasks for the coming sprint.
    
    Alice: I'll finish the frontend redesign. Should be ready for testing by Wednesday next week.
    
    Bob: The API integration needs to be completed as soon as possible. I need help from David's team.
    
    Project Manager: David, can your team support Bob with the API work?
    
    David: Yes, we can help starting Monday. We should be able to complete it within 3 days.
    
    Project Manager: Great. Emily, what about the user documentation?
    
    Emily: I'll have the first draft ready by Friday. I'll need everyone to review it by the middle of next week.
    
    Project Manager: Perfect. Let's reconvene next Thursday to review our progress.
    """
    
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    print(f"Processing transcript from {today}:")
    
    # Extract tasks using our function
    tasks_json = extract_tasks_from_mom(transcript, today)
    tasks = json.loads(tasks_json)
    
    print(f"\nExtracted {len(tasks)} tasks from manual transcript:")
    for i, task in enumerate(tasks, 1):
        print(f"\n{i}. {task['title']}")
        print(f"   Assigned to: {next((tag for tag in task['tags'] if tag.startswith('@')), 'Unassigned')}")
        print(f"   Deadline: {task['deadline'] or 'No deadline'}")
        print(f"   Priority: {task['priority']}")
    
    return tasks

# Process a sample transcript
manual_tasks = process_manual_transcription()